In [145]:
import os
import numpy as np
import pandas as pd
import spark_sklearn as spk

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import GaussianMixture
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.evaluation import BinaryClassificationMetrics

from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType


import matplotlib.pyplot as plt
import pyspark 
os.listdir('./input')

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'installments_payments.csv',
 'previous_application.csv',
 'sample_submission.csv']

In [146]:
from pyspark import SparkContext,SparkConf
sc=SparkContext()

In [147]:
#!wget https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv

In [148]:
data=sc.textFile('./input/application_train.csv')

In [149]:
type(data)

pyspark.rdd.RDD

In [150]:
data.take(2)

['SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APA

In [151]:
spark= SparkSession.builder \
    .master("local") \
    .appName("Kaggele Price") \
    .getOrCreate()

In [152]:
#load data as Spark DataFrame
tr=spark.read.csv("./input/application_train.csv", header='true', inferSchema='true')

In [153]:
te=spark.read.csv("./input/application_test.csv", header='true', inferSchema='true')

In [154]:
tr.count()

307511

In [155]:
#te.show(5)
#fn = F.udf(lambda x: x, DoubleType())
#te.withColumn('AMT_ANNUITY', fn(te.AMT_ANNUITY))

In [156]:
te.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

In [157]:
tr.select(cols).agg(F.max(cols[14])).show()

+-----------------------+
|max(NAME_FAMILY_STATUS)|
+-----------------------+
|                  Widow|
+-----------------------+



In [158]:
tr=tr.dropna()
tr.count()

8602

In [159]:
item = tr.dtypes
str_cols = [i[0] for i in item if i[1] == 'string' ]
str_cols

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [160]:
cols =tr.columns
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(tr) for column in str_cols]

pipeline = Pipeline(stages=indexers)
df_tr = pipeline.fit(tr).transform(tr)

In [161]:
int_cols = [i[0] for i in item  if i[1] == 'int']
int_cols

['SK_ID_CURR',
 'TARGET',
 'CNT_CHILDREN',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [162]:
df_tr.select(cols[20:30]).limit(25).take(4)

[Row(DAYS_ID_PUBLISH=-4176, OWN_CAR_AGE=22.0, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=0, FLAG_EMAIL=0, OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS=2.0),
 Row(DAYS_ID_PUBLISH=-1494, OWN_CAR_AGE=8.0, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=0, FLAG_EMAIL=0, OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS=2.0),
 Row(DAYS_ID_PUBLISH=-2009, OWN_CAR_AGE=4.0, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=0, FLAG_EMAIL=0, OCCUPATION_TYPE='Managers', CNT_FAM_MEMBERS=2.0),
 Row(DAYS_ID_PUBLISH=-4529, OWN_CAR_AGE=3.0, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=1, FLAG_EMAIL=0, OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS=2.0)]

In [163]:
df_tr.select('NAME_CONTRACT_TYPE','CODE_GENDER').show(4)

+------------------+-----------+
|NAME_CONTRACT_TYPE|CODE_GENDER|
+------------------+-----------+
|        Cash loans|          M|
|        Cash loans|          F|
|        Cash loans|          F|
|        Cash loans|          M|
+------------------+-----------+
only showing top 4 rows



In [164]:
transformer = VectorAssembler(inputCols=int_cols[2:],
                              outputCol="features")
standardizer = StandardScaler(withMean=True, withStd=True,
                              inputCol='features',
                              outputCol='std_features')

In [165]:
pca = PCA(k=5, inputCol="std_features", outputCol="pca")
lr = LogisticRegression(featuresCol='std_features', labelCol='TARGET')

pipeline = Pipeline(stages=[transformer, standardizer, pca, lr])


In [166]:
train, validate = df_tr.randomSplit([0.8,0.2],seed=123)

In [167]:
train.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+-------------------+--------------------+--------------+--------

In [168]:
model = pipeline.fit(train)

In [169]:
val_prd = model.transform(validate)

In [170]:
val_prd.select('TARGET','prediction').show(5)

+------+----------+
|TARGET|prediction|
+------+----------+
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
+------+----------+
only showing top 5 rows



In [171]:
validate.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

In [172]:
df_te = model.transform(te)

In [173]:
te.count()

48744

In [174]:
df_te.groupby('prediction').agg(F.count('prediction')).show()

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|       0.0|            39470|
|       1.0|             9274|
+----------+-----------------+



In [175]:
#gmm = GaussianMixture.train(features, k=2)
#prd = gmm.predict(features).collect()
#type(prd)

In [176]:
labels = val_prd.select('TARGET').collect()
prd = val_prd.select('prediction').collect()
#labels = val_prd.select('TARGET').rdd.map(lambda r: r[0]).collect()

In [177]:
np.corrcoef(prd,labels)

/home/ratnesh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3154: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/ratnesh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3088: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
/home/ratnesh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3088: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)


array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [178]:
y_yhat = val_prd.rdd.map(lambda x: (float(x.TARGET), x.prediction))
err = y_yhat.filter(lambda x: x[0] != x[1]).count() / float(val_prd.count())
print("Error = " + str(err))

Error = 0.06053683609366076


In [179]:
val_prd.take(5)

[Row(SK_ID_CURR=100145, TARGET=0, NAME_CONTRACT_TYPE='Cash loans', CODE_GENDER='F', FLAG_OWN_CAR='Y', FLAG_OWN_REALTY='Y', CNT_CHILDREN=1, AMT_INCOME_TOTAL=202500.0, AMT_CREDIT=260725.5, AMT_ANNUITY=16789.5, AMT_GOODS_PRICE=198000.0, NAME_TYPE_SUITE='Family', NAME_INCOME_TYPE='Working', NAME_EDUCATION_TYPE='Secondary / secondary special', NAME_FAMILY_STATUS='Separated', NAME_HOUSING_TYPE='House / apartment', REGION_POPULATION_RELATIVE=0.01885, DAYS_BIRTH=-16282, DAYS_EMPLOYED=-4375, DAYS_REGISTRATION=-762.0, DAYS_ID_PUBLISH=-1494, OWN_CAR_AGE=8.0, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=0, FLAG_EMAIL=0, OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS=2.0, REGION_RATING_CLIENT=2, REGION_RATING_CLIENT_W_CITY=2, WEEKDAY_APPR_PROCESS_START='TUESDAY', HOUR_APPR_PROCESS_START=11, REG_REGION_NOT_LIVE_REGION=0, REG_REGION_NOT_WORK_REGION=0, LIVE_REGION_NOT_WORK_REGION=0, REG_CITY_NOT_LIVE_CITY=0, REG_CITY_NOT_WORK_CITY=0, LIVE_CITY_NOT_WORK_CITY=0, ORGANIZ

In [180]:
# Instantiate metrics object
metrics = BinaryClassificationMetrics(y_yhat)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)


Area under PR = 0.0
Area under ROC = 0.0


In [183]:

from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(y_yhat)

# Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)


/media/ratnesh/SONGS/analytics/spark/python/pyspark/mllib/evaluation.py:237: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")


Summary Stats
Precision = 0.9394631639063392
Recall = 0.9394631639063392
F1 Score = 0.9394631639063392


/media/ratnesh/SONGS/analytics/spark/python/pyspark/mllib/evaluation.py:249: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")
/media/ratnesh/SONGS/analytics/spark/python/pyspark/mllib/evaluation.py:262: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")


In [184]:
# Statistics by class
labels = val_prd.rdd.map(lambda lp: float(lp.TARGET)).distinct().collect()

# Weighted stats
print("Weighted recall = %s" % metrics.weightedRecall)
print("Weighted precision = %s" % metrics.weightedPrecision)
print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)

Weighted recall = 0.9394631639063392
Weighted precision = 1.0
Weighted F(1) Score = 0.9687868080094229
Weighted F(0.5) Score = 0.9872764374024728
Weighted false positive rate = nan


In [189]:
print(labels)

for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))



[0.0, 1.0]
Class 0.0 F1 Measure = 0.9687868080094229


Py4JJavaError: An error occurred while calling o3910.fMeasure.
: java.util.NoSuchElementException: key not found: 1.0
	at scala.collection.MapLike$class.default(MapLike.scala:228)
	at scala.collection.AbstractMap.default(Map.scala:59)
	at scala.collection.mutable.HashMap.apply(HashMap.scala:65)
	at org.apache.spark.mllib.evaluation.MulticlassMetrics.precision(MulticlassMetrics.scala:105)
	at org.apache.spark.mllib.evaluation.MulticlassMetrics.fMeasure(MulticlassMetrics.scala:124)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(A

In [ ]:
sc.stop()